In [1]:
from collections import defaultdict
import gzip
import json

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/johnny/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Load benchmark
with open('swords-v1.1_dev.json', 'r') as f:
    swords = json.load(f)

# Gather substitutes by target
tid_to_sids = defaultdict(list)
for sid, substitute in swords['substitutes'].items():
    tid_to_sids[substitute['target_id']].append(sid)

# Iterate through targets
for tid, target in swords['targets'].items():
    context = swords['contexts'][target['context_id']]
    substitutes = [swords['substitutes'][sid] for sid in tid_to_sids[tid]]
    labels = [swords['substitute_labels'][sid] for sid in tid_to_sids[tid]]
    scores = [l.count('TRUE') / len(l) for l in labels]
    print('-' * 80)
    print(context['context'].replace('\n', ''))
    print('-' * 20)
    print('{} ({})'.format(target['target'], target['pos']))
    print(', '.join(['{} ({}%)'.format(substitute['substitute'], round(score * 100)) for substitute, score in sorted(zip(substitutes, scores), key=lambda x: -x[1])]))
    break

FileNotFoundError: [Errno 2] No such file or directory: 'swords-v1.1_dev.json'

In [ ]:
good_ones = []
for tid, target in swords['targets'].items():
    context = swords['contexts'][target['context_id']]
    substitutes = [swords['substitutes'][sid] for sid in tid_to_sids[tid]]
    labels = [swords['substitute_labels'][sid] for sid in tid_to_sids[tid]]
    scores = [l.count('TRUE') / len(l) for l in labels]
    sorted_scores = sorted(zip(substitutes, scores), key=lambda x: -x[1])
    
    candidate = sorted_scores[0]
    sub = candidate[0]['substitute']
    if candidate[1] > 0.8 and (target['target'].lower() not in sub.lower()) and (' ' not in sub):
        good_ones.append((target['target'].lower(), target['pos'], candidate[0]['substitute']))


In [ ]:
good_ones = list(set(good_ones))

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

pos_mapping = {'NOUN' : 'n', 'VERB' : 'v', 'ADJ' : 'a', 'ADV' : 'r'}
good_ones = [ (lemmatizer.lemmatize(word, pos_mapping[pos]), pos, lemmatizer.lemmatize(substitute, pos_mapping[pos])) \
             for word, pos, substitute in good_ones ]

In [ ]:
good_ones